<html>
    <head>
    </head>
    <body>
        <img src="./jupyter/logo.png" width="500">
    </body>
</html>

In [1]:
import json

from app.widget.widget import select_application

# Lade alle offenen Darlehensanträge
applications = json.load(open('./app/config/application.json', encoding='utf-8'))

# Zeige diese in einer Dropdown-Auswahlliste an
dropdown = select_application(applications)


Bitte wähle einen offenen Antrag aus:


Dropdown(description='Antrag:', options=(('Blanko - Ursula Müller', 'Blanko'), ('Dinglich - Ursula Müller', 'Dinglich')), value='Blanko')

In [ ]:
from app.inform.system import files_uploaded

# Identifiziere aktuelle Dropdown-Auswahl
product = dropdown.value

# Identifiziere den dazugehörigen Darlehensantrag
application = applications[product]

# Identifiziere alle vom Kundenberater übermittelten Dokumente
files = ['./upload/1.jpg', './upload/2.jpg', './upload/3.jpg', './upload/4.jpg', './upload/5.jpg']

# Systeminformation, wenn erfolgreich
files_uploaded(files, product)


In [11]:
from app.extract.document import text_detection
from app.inform.system import documents_extracted

# Extrahiere den Text aus den übermittelten Dokumenten
documents = [text_detection(file) for file in files]

# Systeminformation, wenn erfolgreich
documents_extracted(documents)


SYSTEMNACHRICHT:
Die 4 Unterlagen wurden eingelesen.


In [12]:
from app.decide.document import all_documents, map_to_application
from app.inform.applicant import missing_documents
from app.inform.system import complete_documents

# Ordne übermittelte Dokumente den für den Antrag geforderten Dokumenten zu
application = map_to_application(documents, product)

# Wenn alle geforderten Dokumente übermittelt
if all_documents(application):
    # Systeminformation, wenn alle Dokumente vorhanden
    complete_documents(application, product)

# Ansonsten
else:
    # Kundeninformation, wenn Dokumente fehlen
    missing_documents(application)


AttributeError: 'dict' object has no attribute 'text'

In [ ]:
from app.inform.applicant import show_documents

# Kundeninformation über die Zuordnung der Dokumente
show_documents(application, files)


In [ ]:
from importlib import import_module

from app.inform.system import information_extracted

# Jeder Dokumententyp besitzt eine Funktion zur Extraktion von Informationen
def func(name: str):
    module = import_module('app.extract.information')
    return getattr(module, name)

# Extrahiere für jedes übermittelte Dokumente deren Datenfelder
information = {
    name: func(name.lower())(document['Content']) for name, document in application.items()
}

# Systeminformation, wenn erfolgreich
information_extracted(information)


In [ ]:
from app.decide.document import all_information
from app.inform.applicant import missing_information
from app.inform.system import complete_information

# Überprüfe, ob Dokumente vollständig ausgefüllt
complete = all(all_information(document) for _, document in information.items())

# Wenn alle Dokumente vollständig ausgefüllt
if complete:
    # Systeminformation, wenn alle Informationen vorhanden
    complete_information(information)
    
# Ansonsten
else:
    # Kundeninformation, wenn Informationen fehlen
    missing_information(information)


In [ ]:
from app.decide.information import plausible_person, valid_gehaltsnachweis
from app.inform.applicant import unplausible_information, successful_application

# Überprüfe übermittelte Informationen auf Plausibilität
mismatches = {
    # Überprüfe Konsistenz personenbezogener Daten
    'Person': plausible_person(information), 
    
    # Überprüfe, ob Dokument weniger als einen Monat alt
    'Gehaltsnachweis': valid_gehaltsnachweis(information)
}

# Wenn informationen nicht plausibel
if any(len(mismatch) > 0 for _, mismatch in mismatches.items()):
    # Kundeninformation, wenn Ungereimtheiten im Antrag auftauchen
    unplausible_information(information, mismatches)
    
# Ansonsten
else:
    # Kundeninformation, wenn Antrag plausibel
    successful_application(information, product)
